In [22]:
import os

In [ ]:
%pwd

'd:\\Research\\DL and Computer Vision Project\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC\\research'

: 

: 

In [ ]:
os.chdir("../")

: 

: 

In [ ]:
%pwd

'd:\\Research\\DL and Computer Vision Project\\End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

: 

: 

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/uzzalhasan1238/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="uzzalhasan1238"
os.environ["MLFLOW_TRACKING_PASSWORD"]="da06cc08f11bd6327d4d3759208057fe732dae9c"

: 

: 

In [ ]:
import tensorflow as tf

: 

: 

In [ ]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

: 

: 

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

: 

: 

In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

: 

: 

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/uzzalhasan1238/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




: 

: 

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

: 

: 

In [ ]:
!pip install --upgrade mlflow==2.10.2

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/19.5 MB ? eta -:--:--
    --------------------------------------- 0.3/19.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.5 MB 1.5 MB/s eta 0:00:13
   - -------------------------------------- 0.8/19.5 MB 1.5 MB/s eta 0:00:13
   -- ------------------------------------- 1.0/19.5 MB 1.5 MB/s eta 0:00:13
   -- ------------------------------------- 1.3/19.5 MB 1.5 MB/s eta 0:00:13
   --- ------------------------------------ 1.8/19.5 MB 1.5 MB/s eta 0:00:12
   ---- ----------------------------------- 2.1/19.5 MB 1.5 MB/s eta 0:00:12
   ---- ----------------------------------- 2.4/19.5 MB 1.5 MB/s eta 0:00:12
   ----- ---------------------------------- 2.9/19.5 MB 1.5 MB/s eta 0:00:11
   ------ --------------------------------- 3.1/19.5 MB 1.5 MB/s eta 0:00:11
   ------ -----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.13.2 which is incompatible.


: 

: 

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        # Workaround for MLflow compatibility with TensorFlow 2.12+
        import tensorflow.keras as keras
        if not hasattr(keras, '__version__'):
            keras.__version__ = tf.__version__
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


: 

: 

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2026-01-29 02:25:34,220: INFO: common: yaml file: config\config.yaml loaded successfully]


[2026-01-29 02:25:34,230: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-29 02:25:34,233: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 56s 8s/step - loss: 13.7373 - accuracy: 0.4314
[2026-01-29 02:26:33,191: INFO: common: json file saved at: scores.json]


2026/01/29 02:26:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmp8e5qxiel\model\data\model\assets
[2026-01-29 02:26:41,814: INFO: builder_impl: Assets written to: C:\Users\User\AppData\Local\Temp\tmp8e5qxiel\model\data\model\assets]


2026/01/29 02:26:46 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\User\AppData\Local\Temp\tmp8e5qxiel\model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\User\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2026/01/29 02:28:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


: 

: 